In [1]:
from collections import defaultdict
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [2]:
ee.Authenticate()

Enter verification code: 4/1AZEOvhVVkRGU-fI0qtUMzJ-NrNv6-t2eQqtOGcrtdIuvT4_FLWJJOKmAksk

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
# Global warming potentials from IPCC AR5 Table 8.7. Using GWP-100.
# https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter08_FINAL.pdf
GWP = {
    'CO2': 1,
    'N2O': 265,
    'CH4': 28
}

In [4]:
ch4 = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_CH4_2020')
co2_exclshort = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_CO2exclshort_2020')
co2_short = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_CO2short_2020')
n20 = ee.Image('projects/wri-datalab/SCL-Cities/EDGAR_N2O_2020')
co2e = (co2_exclshort.add(co2_short).add(n20.multiply(GWP['N2O'])).add(ch4.multiply(GWP['CH4']))).multiply(31536) # avg tonnes per sq-m per year

In [5]:
urbext_2020 = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_2020')
urbext_data_2020 = geemap.ee_to_pandas(urbext_2020)

In [7]:
scale = 100

In [10]:
worldpop_2020 = ee.ImageCollection("WorldPop/GP/100m/pop").filter(ee.Filter.equals('year', 2020))
popcodes = worldpop_2020.aggregate_array('country').getInfo()

In [ ]:
emi_res = defaultdict(list)
pop_res = defaultdict(list)
for i in range(len(urbext_data_2020)):
    ua = urbext_data_2020.iloc[i]
    ua_f = urbext_2020.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
    geom = ua_f.geometry()
    if ua['country'] in popcodes:
        localpop_img = worldpop.filter(ee.Filter.equals('country', ua['country'])).select('population').first()
        localemi = co2e.multiply(co2e.pixelArea()).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['b1']
        localpop = localpop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).getInfo()['population']

        emi_res[ua['country']].append(localemi)
        pop_res[emi_res[ua['country']].append(localpop)]
    
    print(i, end=' ')
    
    
out_dicts = []
country_list = list(emi_res.keys())
country_list.sort()
for country in country_list:
    out_dicts.append(
        {
            'country': country,
            'urban_ghg_emissions_tonnesCO2e_2020': sum(emi_res[country]),
            'urban_population_2020': sum(pop_res[country])
        }
    )
res_pd = pd.DataFrame(out_dicts)
res_pd.to_csv('CTY-4_GHGemissions.csv')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 